In [1]:
import pandas as pd
import numpy as np

# Load the data

In [2]:
file_list = pd.read_csv('Data/file_list.csv')

In [3]:
file_list.head()

,file_name,Case_PTC_No,death_date
0,2020-1519BR01.mzML,20201519,6/26/2020
1,2020-1520BG01.mzML,20201520,6/27/2020
2,2020-1521BG01.mzML,20201521,6/25/2020
3,2020-1522BG01.mzML,20201522,6/25/2020
4,2020-1523BG01.mzML,20201523,6/30/2020


In [4]:
tbltest = pd.read_excel('Data/tbltest.xlsx')

In [5]:
tbltest.head()

,Case_PTC_No,Drug_Nm,results.flag
0,20200575,BENZOYLECGONINE,detected
1,20181735,ACETAMINOPHEN,detected
2,20182154,FENTANYL,detected
3,20182062,NORFENTANYL,detected
4,20181851,BENZOYLECGONINE,detected


In [6]:
file_list.shape

(6834, 3)

In [29]:
tbltest['results.flag'].value_counts(dropna=False)

results.flag
detected    109735
Name: count, dtype: int64

In [30]:
tbltest.isnull().sum()

Case_PTC_No     0
Drug_Nm         0
results.flag    0
dtype: int64

In [8]:
tbltest.shape

(109735, 3)

In [9]:
merged_df = pd.merge(file_list, tbltest, on='Case_PTC_No')

In [10]:
merged_df.head()

,file_name,Case_PTC_No,death_date,Drug_Nm,results.flag
0,2020-1519BR01.mzML,20201519,6/26/2020,FENTANYL,detected
1,2020-1519BR01.mzML,20201519,6/26/2020,NORFENTANYL,detected
2,2020-1519BR01.mzML,20201519,6/26/2020,ACETYLFENTANYL,detected
3,2020-1519BR01.mzML,20201519,6/26/2020,METHAMPHETAMINE,detected
4,2020-1519BR01.mzML,20201519,6/26/2020,AMPHETAMINE,detected


In [11]:
merged_df.shape

(45306, 5)

In [12]:
merged_df['results.flag'].value_counts()

results.flag
detected    45306
Name: count, dtype: int64

In [13]:
merged_df['Drug_Nm'].value_counts()

Drug_Nm
FENTANYL                 5482
NORFENTANYL              4468
METHAMPHETAMINE          3203
BENZOYLECGONINE          2830
AMPHETAMINE              2747
                         ... 
3-FLUOROPHENMETRAZINE       1
SAMPLES DISCARDED           1
BUSPIRONE                   1
CARISOPRODOL                1
PARALDEHYDE                 1
Name: count, Length: 234, dtype: int64

In [34]:
merged_df['file_name'].value_counts()

file_name
2021-0997BG01    25
2024-0644BG01    24
2021-0331BG01    24
2021-3677BG01    23
2020-2934BG01    23
                 ..
2021-3435BG01     1
2023-1398BG01     1
2022-2476BG01     1
2022-0922         1
2021-2469BG01     1
Name: count, Length: 6627, dtype: int64

In [14]:
merged_df.to_csv('Data/merged_detected.csv', index=False)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define directories
dates_dir = '/arc/project/st-ashapi01-1/RADD'
rds_dir = '/scratch/st-ashapi01-1/rds_files/data_2024nps-db'
db_dir = '/arc/project/st-ashapi01-1/RADD_libraries'

In [16]:
ms1 = pd.read_csv(os.path.join(rds_dir, 'ms1/ms1.out'))
# Fragments
ms1 = ms1.iloc[:34618]
ms2 = pd.read_csv(os.path.join(rds_dir, 'ms2/ms2.out'))
ms2 = ms2.iloc[:561019]

/tmp/ipykernel_46064/2326074700.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ms2 = pd.read_csv(os.path.join(rds_dir, 'ms2/ms2.out'))


In [17]:
ms1.drop(columns=['spectrum'], inplace=True)

In [18]:
ms2.columns.to_list()

['filename', 'compound_name', 'spectrum', 'm.z', 'mz', 'i']

In [19]:
# Read databases
db1 = pd.read_csv(os.path.join(db_dir, 'NPS_DB-240705.csv'), index_col=False, skiprows=5)
# Remove columns with a single unique value
keep = db1.apply(lambda col: col.nunique() > 1)
db1 = db1.loc[:, keep]

databases = {"NPS": db1}

# Function to calculate PPM range
def calc_ppm_range(theor_mass, err_ppm=10):
    theor_mass = float(theor_mass)  # Ensure theor_mass is float
    lower_bound = (-err_ppm / 1e6 * theor_mass) + theor_mass
    upper_bound = (err_ppm / 1e6 * theor_mass) + theor_mass
    return lower_bound, upper_bound

# Calculate evidence for each peak
def calculate_matches(row):
    lower_bound, upper_bound = calc_ppm_range(row['Product m/z'], err_ppm=20)
    return lower_bound <= row['mz'] <= upper_bound

# Filter fragments and drop duplicates
fragments = db1[db1['Workflow'] == 'Fragment'].drop_duplicates(subset=['Compound Name', 'Product m/z'])

# Calculate evidence for each peak
evidence_per_fragment = ms2.merge(fragments, left_on='compound_name', right_on='Compound Name', how='left')
evidence_per_fragment['match'] = evidence_per_fragment.apply(calculate_matches, axis=1)

# Ensure 'm/z' is included for grouping
evidence_per_fragment = evidence_per_fragment[['filename', 'compound_name', 'spectrum', 'mz', 'Product m/z', 'Retention Time', 'match']]
evidence_per_fragment_original = evidence_per_fragment.copy()
# Group by file, compound, spectrum, and m/z
evidence_per_fragment = (
    evidence_per_fragment
    .groupby(['filename', 'compound_name', 'spectrum', 'mz'])
    .agg(
        match=('match', 'sum'),
        product_mz=('Product m/z', 'first'),
        db_rt=('Retention Time', 'first')  # or use lambda x: x.iloc[0]
    )
    .reset_index()
)

evidence_per_fragment = evidence_per_fragment.reset_index()
# Aggregate evidence per parent
evidence_per_parent = (evidence_per_fragment
                       .groupby(['filename', 'compound_name', 'spectrum'])
                       .agg(matches=('match', 'sum'),
                       db_rt=('db_rt', 'first'))
                       ).reset_index()

# Calculate total fragments
total_fragments = fragments.groupby('Compound Name').size().reset_index(name='total_fragments')

# Merge total_fragments into evidence_per_parent
evidence_per_parent = evidence_per_parent.merge(total_fragments, left_on='compound_name', right_on='Compound Name', how='left')

# Merge with ms1 (including m/z)
ms1 = ms1.merge(evidence_per_parent, on=["filename", "compound_name"], how="left")

# Modify this back to m.z and mz
ms1["mz"] = ms1["mz"].astype(float)
ms1["m.z"] = ms1["m.z"].astype(float)
# Tag ppm error
# mass that was found in the sample compared to precursor ion in the database

# Modify this back to m.z and mz
ms1["ppm_err"] = 1e6 * (ms1["mz"] - ms1["m.z"]) / ms1["m.z"]

In [20]:
# Keep only best evidence per file
best = ms1.sort_values(by=["matches"], ascending=False).groupby(["filename", "compound_name"]).head(1).reset_index(drop=True)
best2 = best[best["matches"] >= 2]

# Calculate proportion of reference product ions detected
best2.loc[:, "prop_reference_ions"] = best2["matches"] / best2["total_fragments"]

/tmp/ipykernel_46064/1646055864.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best2.loc[:, "prop_reference_ions"] = best2["matches"] / best2["total_fragments"]


In [21]:
best2

,filename,compound_name,Retention.Time,Retension.TimeWindow,m.z,mz,rt,intens,spectrum,matches,db_rt,Compound Name,total_fragments,ppm_err,prop_reference_ions
0,2023-2213BG01,Eutylone,NaN,30,236.12812,236.127695,245.673444,3.915749e+06,CP2685.F1.S2959,7,NaN,Eutylone,9,-1.801366,0.777778
1,2024-2266BG01,4'-Cl-deschloroalprazolam,NaN,30,309.09015,309.089996,366.224718,5.001769e+05,CP4600.F1.S4558,6,NaN,4'-Cl-deschloroalprazolam,6,-0.497143,1.000000
2,2023-4588BG01,4'-Cl-deschloroalprazolam,NaN,30,309.09015,309.089539,371.941008,3.206217e+06,CP4230.F1.S4448,6,NaN,4'-Cl-deschloroalprazolam,6,-1.978147,1.000000
3,2023-1175BG01,4'-Cl-deschloroalprazolam,NaN,30,309.09015,309.089752,381.255216,4.291084e+05,CP4485.F1.S4842,6,NaN,4'-Cl-deschloroalprazolam,6,-1.287012,1.000000
4,2024-1573BG01,4'-Cl-deschloroalprazolam,NaN,30,309.09015,309.090179,369.809970,4.704544e+06,CP3545.F1.S4409,6,NaN,4'-Cl-deschloroalprazolam,6,0.095258,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4618,2023-0134BG01,Fluorofentanyl,5.48,10,355.21802,355.217621,335.391300,2.145146e+05,CP1970.F1.S4143,2,5.48,Fluorofentanyl,4,-1.123677,0.500000
4619,2023-0134BG01,Bromazolam,6.37,10,353.03964,353.039185,383.957976,3.761152e+05,CP2154.F1.S4751,2,6.37,Bromazolam,5,-1.290024,0.400000
4620,2023-0391BG01,Desalkylgidazepam,6.26,10,315.01275,315.012329,380.371344,3.464446e+05,CP3408.F1.S4716,2,6.26,Desalkylgidazepam,5,-1.336131,0.400000
4621,2023-0283BG01,Flubromazepam,6.33,10,333.00333,333.002655,378.785142,1.879374e+05,CP3284.F1.S4701,2,6.33,Flubromazepam,3,-2.026919,0.666667


In [22]:
merged_df = pd.read_csv('Data/merged_detected.csv')

In [23]:
merged_df.head()

,file_name,Case_PTC_No,death_date,Drug_Nm,results.flag
0,2020-1519BR01.mzML,20201519,6/26/2020,FENTANYL,detected
1,2020-1519BR01.mzML,20201519,6/26/2020,NORFENTANYL,detected
2,2020-1519BR01.mzML,20201519,6/26/2020,ACETYLFENTANYL,detected
3,2020-1519BR01.mzML,20201519,6/26/2020,METHAMPHETAMINE,detected
4,2020-1519BR01.mzML,20201519,6/26/2020,AMPHETAMINE,detected


In [24]:
merged_df['file_name'] = merged_df['file_name'].str.replace('.mzML', '', regex=False)

In [25]:
merged_df.head()

,file_name,Case_PTC_No,death_date,Drug_Nm,results.flag
0,2020-1519BR01,20201519,6/26/2020,FENTANYL,detected
1,2020-1519BR01,20201519,6/26/2020,NORFENTANYL,detected
2,2020-1519BR01,20201519,6/26/2020,ACETYLFENTANYL,detected
3,2020-1519BR01,20201519,6/26/2020,METHAMPHETAMINE,detected
4,2020-1519BR01,20201519,6/26/2020,AMPHETAMINE,detected


In [26]:
final_df = pd.merge(merged_df, best2, left_on='file_name', right_on='filename')

In [27]:
final_df.head()

,file_name,Case_PTC_No,death_date,Drug_Nm,results.flag,filename,compound_name,Retention.Time,Retension.TimeWindow,m.z,mz,rt,intens,spectrum,matches,db_rt,Compound Name,total_fragments,ppm_err,prop_reference_ions
0,2023-0001BG01,20230001,12/28/2022,COCAINE,detected,2023-0001BG01,Desalkylgidazepam,6.26,10,315.01275,315.012238,379.357932,1.051793e+06,CP4614.F1.S4678,5,6.26,Desalkylgidazepam,5,-1.626763,1.000000
1,2023-0001BG01,20230001,12/28/2022,COCAINE,detected,2023-0001BG01,Bromazolam,6.37,10,353.03964,353.039062,381.541668,2.304417e+05,CP4607.F1.S4669,5,6.37,Bromazolam,5,-1.635794,1.000000
2,2023-0001BG01,20230001,12/28/2022,COCAINE,detected,2023-0001BG01,Fluorofentanyl,5.48,10,355.21802,355.217497,325.845654,3.380081e+05,CP4173.F1.S4003,4,5.48,Fluorofentanyl,4,-1.471573,1.000000
3,2023-0001BG01,20230001,12/28/2022,COCAINE,detected,2023-0001BG01,3-methylmethcathinone,NaN,30,178.12264,178.122612,303.849222,8.334922e+05,CP5237.F1.S3987,3,NaN,3-methylmethcathinone,7,-0.156422,0.428571
4,2023-0001BG01,20230001,12/28/2022,COCAINE,detected,2023-0001BG01,Deschloroetizolam,6.16,10,309.11684,309.116393,370.940448,1.766447e+05,CP4519.F1.S4570,2,6.16,Deschloroetizolam,2,-1.445459,1.000000


In [28]:
final_df.to_csv('Data/final_df_with_best2.csv', index=False) 

In [32]:
final_df['results.flag'].value_counts()

results.flag
detected    35696
Name: count, dtype: int64

In [33]:
final_df.shape

(35696, 20)